In [46]:
import random as r
import itertools

In [47]:
class Bacteria:
    def __init__(self, R=None, G=None, B=None, target=[96, 96, 159]):
        if not all([R, G, B]):
            self.__R = r.randint(0, 256)
            self.__G = r.randint(0, 256)
            self.__B = r.randint(0, 256)
        else:
            self.__R = R
            self.__G = G
            self.__B = B
        
        self.__RT = target[0]
        self.__GT = target[1]
        self.__BT = target[2]
            
        self.__pm = 0.7
        
        
    def _evaluete(self):
        return ((self.__RT - self.__R) ** 2 + (self.__GT - self.__G) ** 2 + (self.__BT - self.__B) ** 2) ** .5

    
    def __add__(self, other):
        gen = r.randint(0, 2)
        
        genom_1 = self()
        genom_2 = other()
        
        gen_1 = genom_1.pop(gen)
        gen_2 = genom_2.pop(gen)
        
        new_gen = int("{0:b}".format(gen_1).zfill(4)[:4] + "{0:b}".format(gen_2).zfill(8)[4:8], 2)
        combinations = [list(comb) for comb in itertools.product(genom_1, genom_2)]
        
        mutated_bact = []
        for comb in combinations:
            comb_c = comb.copy()
            if self.__pm > r.random():
                mutated_index = r.randint(0, 1)
                mutated = "{0:b}".format(comb[mutated_index]).zfill(8)
                
                bit_indices = r.sample(range(len(mutated)), r.randint(4, 8))
                inverted_num = ''.join([str((int(bit, 2) + 1) % 2) if i in bit_indices else bit for i, bit in enumerate(mutated)]) 
                
                comb_c[mutated_index] = int(inverted_num, 2)
            mutated_bact.append(comb_c)
            
        final_out = []
        for comb in mutated_bact:
            comb_c = comb.copy()
            comb_c.insert(gen, new_gen)
            
            final_out.append(comb_c)
        
        return [Bacteria(*i, target=[self.__RT, self.__GT, self.__BT]) for i in final_out]
    
    def __gt__(self, other):
        return self._evaluete() < other._evaluete()


    def __call__(self):
        return [self.__R, self.__G, self.__B]

In [48]:
def tournament_selection(population):
    selected_parents = []
    
    for _ in range(len(population) // 3):
        top = min(r.sample(population, 4), key=Bacteria._evaluete)
        selected_parents.append(top)

    return selected_parents

In [49]:
def top_fitness(population):
    return sorted([round(x._evaluete(), 3) for x in population])[:3]

In [50]:
len_start_prop = 80
start_pop = [Bacteria(target=[96, 96, 159]) for i in range(len_start_prop)]
selected = tournament_selection(start_pop)

counter = 0

while True:
    counter += 1
    
    for prop in selected:
        if prop._evaluete() == 0:
            print(f"Epoch = {counter}, Finded = {prop()}")
            break
    else:
        new_pop = []
        for i in range(len(selected)):
            if 0.6 > r.random():
                new_pop.extend(selected[i-1] + selected[i])
            else:
                new_pop.append(selected[i-1])
                new_pop.append(selected[i])

        selected = tournament_selection(new_pop)
        if counter % 20 == 0:
            print(f"Epoch = {counter}", f"\nTop Fitness: {top_fitness(selected)}")
            selected = sorted(selected, key=Bacteria._evaluete)[:len_start_prop]
        continue
    break

Epoch = 20 
Top Fitness: [4.243, 4.243, 4.243]
Epoch = 40 
Top Fitness: [3.0, 3.0, 3.0]
Epoch = 60 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 80 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 100 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 120 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 140 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 160 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 180 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 200 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 220 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 240 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 260 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 280 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 300 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 320 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 340 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 360 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 380 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 400 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 420 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 440 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 460 
Top Fitness: [1.0, 1.0, 1.0]
Epoch = 480 
Top Fitness: [1.0, 